In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [ ]:
print(os.listdir("../input"))
df = pd.read_csv("../input/911.csv")

** Some initial analysis of the dataset **

In [ ]:
df.info()

In [ ]:
df.head()

**What are the top 5 zipcodes for 911 calls?**

In [ ]:
df['zip'].value_counts().head(5)

** What are the top 5 townships (twp) for 911 calls? **

In [ ]:
df['twp'].value_counts().head(5)

** How many unique title codes are there? **

In [ ]:
df['title'].nunique()

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. I will cretae a custom new column called "Reason" that contains this value.** 

In [ ]:
df['Reason'] = df['title'].apply(lambda title: title.split(':')[0])
df['Sub_Reason'] = df['title'].apply(lambda title: title.split(':')[1])

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
df['Reason'].value_counts()

** Now we can create a countplot of 911 calls by Reason. 
We can see there was significantly less Fire related calls than EMS anf Traffic.**

In [ ]:
sns.countplot(x = df['Reason'])

In [ ]:
sns.countplot(x = df['Sub_Reason'])

**The data type of the objects in the timeStamp column is a string rather than a date data type **

In [ ]:
df.info()


** I will change the data type to datetime**

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
type(df['timeStamp'].iloc[0])

** Now i can use DateTime attributes and methods. I will create 3 new columns called Hour, Month, and Day of Week based off of the timeStamp column.**

In [ ]:
df['timeStamp'].dt.hour
df['hour'] = df['timeStamp'].apply(lambda timeStamp:timeStamp.hour)
df['month'] = df['timeStamp'].apply(lambda timeStamp:timeStamp.month)
df['day_of_week'] = df['timeStamp'].apply(lambda timeStamp:timeStamp.weekday())

In [ ]:
df[['timeStamp','hour','month' ,'day_of_week']].head()

** The day of week filed is an integer data type. I will use the map() function together with the following  dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
df['week_day']= df['day_of_week'].map(dmap)


** Now we can plot the 911 calls based on the day during the week the call was placed  - there seems to be less 911 calls on Sunday**

In [ ]:
        sns.countplot(df['week_day'],palette='viridis')

** Now the same plot, with the call "Reason" added **

In [ ]:
sns.countplot(df['week_day'], hue = df['Reason'],palette='viridis')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

** The same for Month:**

In [ ]:
sns.countplot(df['month'], hue = df['Reason'],palette='viridis')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

**Did you notice something strange about the Plot?**

_____

** You should have noticed it was missing some Months, let's see if we can maybe fill in this information by plotting the information in another way, possibly a simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas... **

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
byMonth = df.groupby(by='month').count()
#byMonth is a dataframe
byMonth

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
byMonth['lat'].plot()

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
sns.lmplot(data = byMonth.reset_index(), x = 'month', y='lat', size=4, aspect = 2)
plot = plt.gca()
plot.set_title("calls per month")
plt.ylabel('count')